## Лекция 2  BM5    

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [274]:
all = []
numb = 0
import csv
with open('quora_question_pairs_rus.csv') as qd:
    qd = csv.reader(qd, delimiter=',')
    for raw in qd:
        numb += 1
        if numb < 500 and numb >= 2:
            all.append(raw)

In [14]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk
import collections
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayakorotkaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import re

In [275]:
normal = []
for i in all:
    normal_pair = []
    normal_query = []
    normal_doc = []
    for t in tokenizer.tokenize(i[1]):
        t = morph.parse(t)[0]
        if t.normal_form not in russian_stopwords and t.normal_form != 'это' and t.normal_form != 'весь' and not re.match(r'[0-9A-Za-z]', t.normal_form):
            normal_query.append(t.normal_form)
    normal_pair.append(normal_query)
    for k in tokenizer.tokenize(i[2]):
        k = morph.parse(k)[0]
        if k.normal_form not in russian_stopwords and k.normal_form != 'это' and k.normal_form != 'весь' and not re.match(r'[0-9A-Za-z]', k.normal_form):
            normal_doc.append(k.normal_form)
    normal_pair.append(normal_doc)
    normal_pair.append(i[3])
    normal.append(normal_pair)

In [ ]:
all

In [256]:
normal

[[['каков', 'история', 'кохинор', 'кох', 'ноор', 'бриллиант'],
  ['произойти',
   'правительство',
   'индия',
   'украсть',
   'кохинор',
   'кох',
   'ноор',
   'алмаз',
   'назад'],
  '0'],
 [['мочь', 'увеличить', 'скорость', 'интернет', 'соединение', 'использовать'],
  ['повысить', 'скорость', 'интернет', 'путём', 'взлом'],
  '0'],
 [['почему', 'мысленно', 'очень', 'одинокий', 'мочь', 'решить'],
  ['найти', 'остаток', 'математика', 'математика', 'разделить'],
  '0'],
 [['который',
   'растворять',
   'вода',
   'быстро',
   'сахарный',
   'соль',
   'метан',
   'углеродный',
   'диоксид'],
  ['рыба', 'выжить', 'солёный', 'вода'],
  '0'],
 [['астрология',
   'луна',
   'колпачок',
   'козерог',
   'крышка',
   'поднимать',
   'говорить'],
  ['тройной', 'луна', 'козерог', 'восхождение', 'козерог', 'говорить'],
  '1'],
 [['должный', 'купить'],
  ['делать', 'ребёнок', 'активный', 'далёкий', 'телефонный', 'видеоигра'],
  '0'],
 [['мочь', 'хороший', 'геолог'],
  ['должный', 'делать', 'ве

In [276]:
### реализуйте эту функцию ранжирования 
from math import log

k = 2.0
b = 0.75
N = len(all)

l = 0
for i in normal:
    l += len(i[1])
avgdl = l/N

def bm25(D, Q) -> float:
    score = 0
    for q in Q:
        n = 0
        for doc in normal:
            if q in doc[1]:
                n += 1
        IDF = log((N - n + 0.5)/(n+0.5))
        TF = 0
        for word in D:
            if word == q:
                TF += 1
        TF = TF/len(D)
        score += IDF*((TF*(k + 1))/(TF + (k*(1 - b + (b*(len(D)/avgdl))))))
    return score

In [277]:
bm25(normal[4][1], normal[4][0])

4.666286533192796

In [278]:
bm = {}

In [207]:
from operator import itemgetter

In [280]:
for z in range(len(all)):
    band = {}
    for i in range(len(all)):
        try:
            band[all[i][2]] = bm25(normal[i][1], normal[z][0])
        except ZeroDivisionError:
            pass
    query = all[z][1]
    bm[query] = sorted(band.items(), key=itemgetter(1))[-10:]

In [288]:
bm

{'5 фактов о земной коре': [('в чем разница между астероидами и кометами',
   0.0),
  ('почему нерафинированное молоко хорошо для вас', 0.0),
  ('какие уловки для эффективного изучения', 0.0),
  ('которая является лучшей страной для высшего образования после США и Великобритании',
   0.0),
  ('что это за картина?', 0.0),
  ('которые являются основными автомагистралями в Калифорнии и как они сравниваются с основными магистралями в техасе',
   0.0),
  ('если пространство расширяется, где новое пространство исходит из', 0.0),
  ('может лысый человек когда-либо вырастить свои волосы назад', 0.0),
  ('как земное ядро \u200b\u200bвлияет на его корку', 2.400088683911345),
  ('какие интересные факты о Facebook', 5.683407772417998)],
 'daniel ek: когда мы ожидаем выявить в Индии': [('почему Индия не применяет модель burma-rohingya для депортации незаконных бангладеш',
   0.6130685243427562),
  ('как демонизация может повлиять на gdp Индии как в краткосрочной, так и в долгосрочной перспективе',


In [281]:
scores = 0
for key in bm:
    for i in all:
        if key == i[1] and i[3] == '1':
            for z in bm[key]:
                if z[0] == i[2]:
                    scores += 1

In [283]:
print('Настолько верно для 500 запросов: ',scores*100/len(bm), '%')

Настолько верно для 500 запросов:  35.010060362173036 %


### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [293]:
bmCh = {}
christmas = ['рождественский', 'каникулы']
temp = {}
for t in range(len(all)):
    try:
        temp[all[t][2]] = bm25(normal[t][1], christmas)
    except ZeroDivisionError:
        pass
bmCh['рождественские каникулы'] = sorted(temp.items(), key=itemgetter(1))[-10:]

In [294]:
bmCh

{'рождественские каникулы': [('у rahul gandhi есть шансы стать следующим pm of india после modi',
   0.0),
  ('хорошо делать спортзал утром и плавать в полдень', 0.0),
  ('в чем разница между астероидами и кометами', 0.0),
  ('почему нерафинированное молоко хорошо для вас', 0.0),
  ('какие уловки для эффективного изучения', 0.0),
  ('которая является лучшей страной для высшего образования после США и Великобритании',
   0.0),
  ('что это за картина?', 0.0),
  ('которые являются основными автомагистралями в Калифорнии и как они сравниваются с основными магистралями в техасе',
   0.0),
  ('если пространство расширяется, где новое пространство исходит из', 0.0),
  ('может лысый человек когда-либо вырастить свои волосы назад', 0.0)]}

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1